# Chinook music store: SQL project

In this project, our goal is to analyze the Chinook database, a fictional digital music store, and provide insights related to various business tasks. We will take a concise and straightforward approach to communicate our findings effectively.

First, we will select the first three albums to be added to the store from a new record label, considering sales by genre in the USA.

Next, we will evaluate the performance of sales support agents by analyzing customer purchases. Specifically, we will examine whether any agent outperforms or underperforms compared to others.

Furthermore, we will analyze sales data from different countries to understand trends.

Lastly, we will assess the impact of changing the purchasing strategy on overall revenue. Specifically, we will determine the percentage of individual track purchases versus whole album purchases.

By following a concise and matter-of-fact approach, we aim to provide a quick and informative read that presents the project's goal, approach, and key findings clearly.

In [18]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Take a look at the database's tables.

In [19]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase

Suppose the Chinook record store has just signed a deal with a new record label, and we've been given the task of selecting the first three albums to be added to the store from a list of four. All four albums are by artists who don't currently have any tracks in the store - we have the artist names and the genre of music they produce:
- Regal (Hip-Hop)
- Red Tone (Punk)
- Meteor and the Girls (Pop)
- Slim Jim Bites (Blues)

The record company specializes in artists from the USA, and they've given Chinook some money to promote the new albums in the USA, so we're interested in finding out which genres sell best in the USA.

We will find out which genres sell the most in the US and make a recommendation for the three artists whose albums we should buy for the store.

In [3]:
%%sql
WITH
    i  AS (SELECT
               il.track_id,
               il.quantity
            FROM invoice AS inv
            INNER JOIN invoice_line AS il
            ON inv.invoice_id = il.invoice_id
            WHERE billing_country = 'USA'),
    g  AS (SELECT
               genre_id,
               name
           FROM genre)
SELECT
    g.name AS genre,
    COUNT(t.track_id) AS tracks_sold,
    ROUND(
          CAST(COUNT(t.track_id) AS float) * 100 / (SELECT
                                                        CAST(SUM(i.quantity) AS float)
                                                        FROM i),
          2) AS tracks_sold_pct
FROM track AS t
INNER JOIN i ON i.track_id = t.track_id
INNER JOIN g ON g.genre_id = t.genre_id
GROUP BY g.name
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


genre,tracks_sold,tracks_sold_pct
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on sales by genre in the U.S., we would recommend buying Red Tone (punk), Meteor and the Girls (pop), and Slim Jim Bites (blues) albums from a new label.

## Analyzing Employee Sales Performance

Each customer for the Chinook store is assigned to a Sales Support Agent within the company when they first make a purchase. We will analyze the purchases of each employee's customers to see if one sales support agent is performing better or worse than the others.

In [4]:
%%sql
SELECT
    e.first_name || " " || e.last_name AS name,
    title,
    ROUND(SUM(total), 2) AS sales
FROM employee AS e
LEFT JOIN customer AS c ON c.support_rep_id = e.employee_id
LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
GROUP BY 1
ORDER BY 3 DESC

 * sqlite:///chinook.db
Done.


name,title,sales
Jane Peacock,Sales Support Agent,1731.51
Margaret Park,Sales Support Agent,1584.0
Steve Johnson,Sales Support Agent,1393.92
Robert King,IT Staff,None
Nancy Edwards,Sales Manager,None
Michael Mitchell,IT Manager,None
Laura Callahan,IT Staff,None
Andrew Adams,General Manager,None


We have a clear division between our Sales Support Agents, it's worth investigating further what might be the cause of this. We will take a look at the country distribution of sales by each agent.

In [5]:
%%sql
SELECT
    e.first_name || " " || e.last_name AS name,
    ROUND(SUM(total), 2) AS sales,
    billing_country AS country
FROM employee AS e
LEFT JOIN customer AS c ON c.support_rep_id = e.employee_id
LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
WHERE e.title = 'Sales Support Agent'
GROUP BY 3
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


name,sales,country
Jane Peacock,1040.49,USA
Jane Peacock,535.59,Canada
Jane Peacock,427.68,Brazil
Jane Peacock,389.07,France
Jane Peacock,334.62,Germany
Margaret Park,273.24,Czech Republic
Jane Peacock,245.52,United Kingdom
Margaret Park,185.13,Portugal
Jane Peacock,183.15,India
Jane Peacock,114.84,Ireland


Jane Peacock's huge turnover is due to the fact that she is responsible for the most populous and economically developed countries - the USA, Canada, Brazil, France, Germany and the UK - while her colleagues share the remaining territories.

## Analyzing Sales by Country

Suppose our next task is to analyze the sales data for customers from each different country. We will use the country value from the customer table and ignore the country from the billing address in the invoice table.

In particular, we will calculate data for each country on
- Total number of customers
- Total value of sales
- Average sales value per customer
- Average order value.

In [6]:
%%sql
SELECT  c.country,
        COUNT(DISTINCT c.customer_id) AS uniq_count,
        SUM(i.total) AS sales
FROM invoice AS i
INNER JOIN customer AS c ON i.customer_id = c.customer_id
GROUP BY c.country

 * sqlite:///chinook.db
Done.


country,uniq_count,sales
Argentina,1,39.6
Australia,1,81.18
Austria,1,69.3
Belgium,1,60.38999999999999
Brazil,5,427.68000000000006
Canada,8,535.5900000000001
Chile,1,97.02000000000001
Czech Republic,2,273.24000000000007
Denmark,1,37.61999999999999
Finland,1,79.2


In [7]:
%%sql
WITH
    sales_1 AS (SELECT 
                *,
                COUNT(DISTINCT c.customer_id) AS uniq_customers,       -- counting unique customers
                SUM(i.total) AS sales,
                COUNT(i.invoice_id) AS orders
            FROM invoice AS i
            INNER JOIN customer AS c ON i.customer_id = c.customer_id  -- joining customer and invoice tables
            GROUP BY c.country),
    sales_2 AS (SELECT 
                CASE                                                   -- countries with 1 customer
                    WHEN uniq_customers = 1 THEN 'Other'               -- will be labeled as 'Other'
                    ELSE sales_1.country
                    END AS country,
                customer_id,
                invoice_id,
                sales,
                uniq_customers,
                orders
                FROM sales_1)
SELECT
    country,
    ROUND(SUM(sales), 2) AS sales,
    SUM(uniq_customers) AS number_of_customers,
    ROUND(SUM(sales)/SUM(uniq_customers), 2) AS avg_sales_per_customer,
    ROUND(SUM(sales)/SUM(orders), 2) AS avg_order_value
FROM 
    (SELECT
     s_2.*,
     CASE                                                              -- 'Other' countries will be at the bottom
         WHEN s_2.country = 'Other' THEN 1
         ELSE 0
     END AS sort
     FROM sales_2 AS s_2)
GROUP BY country
ORDER BY sort, sales DESC

 * sqlite:///chinook.db
Done.


country,sales,number_of_customers,avg_sales_per_customer,avg_order_value
USA,1040.49,13,80.04,7.94
Canada,535.59,8,66.95,7.05
Brazil,427.68,5,85.54,7.01
France,389.07,5,77.81,7.78
Germany,334.62,4,83.66,8.16
Czech Republic,273.24,2,136.62,9.11
United Kingdom,245.52,3,81.84,8.77
Portugal,185.13,2,92.57,6.38
India,183.15,2,91.58,8.72
Other,1094.94,15,73.0,7.45


The majority of sales come from the USA, Canada and Brazil, while 15 countries with a single customer together account for approximately 1/4 of total sales.

Average revenue per customer and average order value are highest in the Czech Republic.

## Albums vs Individual Tracks

The Chinook store is set up to allow customers to make purchases in one of two ways:
- purchase an entire album
- a collection of one or more individual tracks.

The store does not allow customers to purchase an entire album and then add individual tracks to the same purchase (unless they do so by manually selecting each track). When customers purchase albums, they are charged the same price as if they had purchased each of those tracks individually.

Let's say that management is currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from the record companies, instead of purchasing every track from an album.

We will find out what percentage of purchases are individual tracks versus entire albums, so that management can use this data to understand the impact of this decision on overall revenue.

In [75]:
%%sql
WITH
    t1 AS (
                SELECT
                    COUNT(DISTINCT t.album_id) AS album_count,
                    COUNT(DISTINCT i.track_id) AS track_count,
                    album_id,
                    invoice_id
                FROM invoice_line AS i
                INNER JOIN track AS t ON t.track_id = i.track_id
                GROUP BY invoice_id
                HAVING album_count = 1),
-- selecting invoices where tracks purchased are from single album, and counting tracks
    t2 AS (
                SELECT
                    a.album_id AS album_id,
                    COUNT(t.track_id) AS track_count
                FROM album AS a
                INNER JOIN track AS t ON t.album_id = a.album_id
                GROUP BY a.album_id),
-- selecting all albums and counting tracks for each album
    t3 AS (
                SELECT
                    t1.invoice_id,
                    CASE
                        WHEN t1.track_count = t2.track_count THEN 1
                        END AS alb_cat -- creating dummy variable
                FROM t1
                LEFT JOIN t2 ON t2.album_id = t1.album_id
                WHERE t1.track_count = t2.track_count),
-- selecting invoices (t1) where number of tracks per invoice equals number of tracks per album (t2)
    t4 AS (
                SELECT * FROM t3

                UNION

                SELECT
                    invoice_id,
                    CASE
                        WHEN billing_city = 0 THEN 0
                        ELSE 0
                        END AS alb_cat -- creating dummy variable
                FROM invoice),
-- combining t3 and list of all invoices
    t5 AS (
                SELECT 
                    invoice_id,
                    SUM(alb_cat) AS alb_cat
                FROM t4
                GROUP BY 1)
-- creating final table, where SUM(alb_cat) equals (0 + 0) if invoice was in t3,
-- and equals (0 + 1) otherwise
SELECT
    CASE
        WHEN alb_cat = 0 THEN 'single purchase'
        ELSE 'album purchase'
        END AS purchase_type,
    COUNT(invoice_id) AS invoices,
    ROUND(
        COUNT(invoice_id) / CAST((SELECT
                            COUNT(*) FROM invoice) AS float) * 100,
        2) AS invoices_pct
FROM t5
GROUP BY alb_cat

 * sqlite:///chinook.db
Done.


purchase_type,invoices,invoices_pct
single purchase,500,81.43
album purchase,114,18.57


The majority of customers (over 80%) buy individual tracks rather than whole albums, the potential revenue impact of switching to track-by-track purchases from record companies will be 20% at most, so it is worth investigating further how substantial a cost reduction such a move would be.

## Conclusion

In conclusion, our project aimed to analyze the Chinook database, a fictional digital music store, to address various business tasks. We followed a concise and systematic approach to achieve our goals.

First, we selected the first three albums from a new record label based on sales by genre in the USA. Our recommendation includes the Red Tone (punk), Meteor and the Girls (pop), and Slim Jim Bites (blues) albums, leveraging the potential for advertisement and sales in the USA market.

Additionally, we evaluated the performance of sales support agents by examining customer purchases. Jane Peacock stood out with remarkable turnover, attributed to her responsibility for the most populous and economically developed countries. Her colleagues, on the other hand, handled the remaining territories effectively.

Furthermore, we analyzed sales data from different countries and uncovered valuable insights. The USA, Canada, and Brazil emerged as the major contributors to overall sales, while 15 countries with a single customer collectively accounted for a significant portion of revenue. Notably, the Czech Republic exhibited the highest average revenue per customer and average order value.

Lastly, we assessed the impact of changing the purchasing strategy on overall revenue. Our analysis revealed that the majority of customers (over 80%) prefer buying individual tracks rather than entire albums. Therefore, implementing a strategy of purchasing only popular tracks could result in a potential revenue impact of 20% at most.

By adopting a concise and data-driven approach, we successfully achieved our project goals of providing valuable recommendations for album selection, evaluating sales support agents' performance, understanding sales trends by country, and assessing the impact of a new purchasing strategy. These findings offer actionable insights for the Chinook music store to optimize its operations and drive business growth.